In [ ]:
from PatientDataset import group_split_df, transforms, PatientDS, get_df
from Train import train, plot_train_hist, set_seed_mps, patient_multiclass_collate, get_sampler
from Evaluate import evaluate
from Models import ResNet101Meta_19_20, Attention_Pool, PatientCNNClassifier
from torch.utils.data import DataLoader
import torch
import torch.nn as nn

In [9]:
# Load and Preprocess Data
train_transform, val_transform, test_transform = transforms(img_size=224)

train_ds = PatientDS(train_transform, image_path = 'train/2020_jpg', use_meta=True, upsampling = False, mode='Train', saving_images=False)
val_ds = PatientDS(val_transform, image_path = 'train/2020_jpg', use_meta=False, mode='val', saving_images=False)
test_ds = PatientDS(test_transform, image_path = 'train/2020_jpg', use_meta=False, mode='test', upsampling = False, saving_images=False)

sampler, pos_w = get_sampler(train_ds)

train_dl = DataLoader(train_ds, batch_size=1,  sampler=sampler, shuffle = False, num_workers=0)
val_dl = DataLoader(val_ds, batch_size=1,  sampler=None,  shuffle=False, num_workers=0)
test_dl = DataLoader(test_ds, batch_size=1,  sampler=None, shuffle=False, num_workers=0)

In [10]:
x, y, m = next(iter(train_dl))
x ,y, m = x.squeeze(0).to('mps'), y.squeeze(0).to('mps'), m.squeeze(0).to('mps')
x.shape, y.shape, m.shape

(torch.Size([12, 3, 224, 224]), torch.Size([4]), torch.Size([12, 3]))

In [4]:
resnet = ResNet101Meta_19_20()
resnet.to('mps')

logits = resnet(x,m)
logits.shape

torch.Size([23, 9])

In [5]:
attention = Attention_Pool(embed_size=9, new_embed = 4, num_heads=1, attention_type="self")
attention.to('mps')
out = attention(logits)
out.shape

torch.Size([1, 4])

In [6]:
net = PatientCNNClassifier()
net.to('mps')
logits = net(x, m)
logits.shape

torch.Size([4])

In [7]:
y.shape

torch.Size([4])

In [11]:
# Set up Model
# criterion = nn.BCEWithLogitsLoss() # without weighted loss
net = PatientCNNClassifier()
net.to('mps')
pos_weight = torch.tensor([57*0.4, 1, 1, 1], device='mps', dtype = torch.float32)
criterion = nn.BCEWithLogitsLoss(pos_weight= pos_weight) # with weighted loss
opt = torch.optim.AdamW([
    {"params": net.attention.parameters(), "lr": 1e-4},
])

# Train only the last layers (freeze the backbone)
for p in net.attention.parameters():
    p.requires_grad = True
for p in net.resnet.parameters():
    p.requires_grad = False
for p in net.resnet.layer4.parameters():
    p.requires_grad = True
for p in net.resnet.head.parameters():
    p.requires_grad = True

hist_train_loss_res, hist_val_loss_res, hist_val_acc_res = train(net, train_dl, val_dl, criterion, opt,
                                                     epochs = 1, best_model_name = "testing.pt")

100%|██████████| 1644/1644 [41:12<00:00,  1.50s/it] 


Saved new best model at epoch 1 with AUC=0.6855
Epoch 1: Val ACC = 0.6893203883495146 Loss Ratio=0.8247, Val AUC=0.6855, Val Pos Acc=0.3304
Best validation pos_acc: 0.3304
